In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gc
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('./Binance_BTCUSDT_1h.csv', date_parser = True)
df = df.reindex(-np.sort(-np.arange(len(df))))
len(df)

40424

In [3]:
#擷取⽇期及收盤價
df2 = df.loc[:, ('date','close', 'open','high','low','Volume BTC')] ; del df ; gc.collect()
#切出⽇期時間欄位，符號統⼀
df2['NewDate'] = df2.loc[:, 'date'].str.slice(start = 0, stop = 10).str.replace('-', '/').str.replace('/0', '/').str.rsplit(pat=' ', n=1).apply(lambda x : x[0])
#判斷⽇期欄位格式
df2['AMPM'] = df2.loc[:, 'date'].str.contains(pat = 'AM|PM')
#依⽇期欄位格式切分兩分DF，有AMPM的在df3，正常格式的在df4
df3 = df2[df2['AMPM'] == 1]
df4 = df2[df2['AMPM'] == 0]
del df2 ; gc.collect()


48

In [4]:
#切出⽇期欄位中的⼩時
df3['Hour'] = df3.loc[:, 'date'].str.slice(start = 10, stop = 13).str.replace(':', '').apply(lambda x : pd.to_numeric(x))
df3['AMPM'] = df3.loc[:, 'date'].str.slice(start = 14, stop = 16).apply(lambda x: 12 if (x == 'PM') else 0)
df3['24Hr'] = (df3['Hour'] + df3['AMPM'])
#轉換為24⼩時制
df3['24Hr'] = df3['24Hr'].apply(lambda x: (x == 24 and 12) or (x == 12 and '00') or x)
df3['24Hr'] = df3['24Hr'].apply(lambda x: '0'+str(x) if len(str(x)) == 1 else str(x))

#轉為時間格式
df3['NewDateTime'] = (df3['NewDate'] + ' ' +df3['24Hr'])
df3['NewDateTime'] = df3['NewDateTime'].apply(lambda x: pd.to_datetime(x, format = "%Y/%m/%d %H"))
#移除重複
df3 = df3[['close', 'open','high','low','Volume BTC', 'NewDateTime','NewDate']].drop_duplicates()
df3.head()

,close,open,high,low,Volume BTC,NewDateTime,NewDate
40423,4308.83,16199.91,16199.91,4261.32,44.5100,2017-08-17 04:00:00,2017/8/17
40422,4315.32,4308.83,4328.69,4291.37,23.2300,2017-08-17 05:00:00,2017/8/17
40421,4324.35,4315.32,4345.45,4309.37,7.2300,2017-08-17 06:00:00,2017/8/17
40420,4349.99,4324.35,4349.99,4287.41,4.4400,2017-08-17 07:00:00,2017/8/17
40419,4360.69,4349.99,4377.85,4333.32,0.9499,2017-08-17 08:00:00,2017/8/17


In [5]:
#轉為時間格式並移除重複
df4['NewDateTime'] = df4['date'].apply(lambda x: pd.to_datetime(x, format = "%Y/%m/%d %H:%M"))
df4 = df4[['close', 'open','high','low','Volume BTC', 'NewDateTime','NewDate']].drop_duplicates()
df4.head()

,close,open,high,low,Volume BTC,NewDateTime,NewDate
14487,11801.17,11783.31,11820.61,11731.71,2194.598589,2020-08-01 23:00:00,2020/8/1
14485,11782.63,11801.17,11857.00,11740.00,2872.638184,2020-08-02 00:00:00,2020/8/2
14483,11942.55,11782.63,11942.55,11772.13,3707.136801,2020-08-02 01:00:00,2020/8/2
14481,11928.58,11942.54,11965.01,11892.88,3785.415559,2020-08-02 02:00:00,2020/8/2
14479,12000.00,11928.40,12000.00,11870.00,3746.422743,2020-08-02 03:00:00,2020/8/2


In [6]:
#合併df3、df4，並移除重複
df5 = pd.concat([df3, df4]) ; df6 = df5.drop_duplicates()
#確認刪除多少資料
print(len(df5), len(df6))
del (df3, df4, df5)
gc.collect()


40424 40424


7

In [7]:
#投報率
df6['returns']=df6['close']/df6['close'].shift(1)
df6['log_returns']=np.log(df6['returns'])
df6 = df6[df6['returns'].notna()]
# df6.dropna()
df6.head()

,close,open,high,low,Volume BTC,NewDateTime,NewDate,returns,log_returns
40422,4315.32,4308.83,4328.69,4291.37,23.2300,2017-08-17 05:00:00,2017/8/17,1.001506,0.001505
40421,4324.35,4315.32,4345.45,4309.37,7.2300,2017-08-17 06:00:00,2017/8/17,1.002093,0.002090
40420,4349.99,4324.35,4349.99,4287.41,4.4400,2017-08-17 07:00:00,2017/8/17,1.005929,0.005912
40419,4360.69,4349.99,4377.85,4333.32,0.9499,2017-08-17 08:00:00,2017/8/17,1.002460,0.002457
40418,4444.00,4360.69,4445.78,4360.00,10.7600,2017-08-17 09:00:00,2017/8/17,1.019105,0.018925


In [8]:
#依照年份及週次分組
df6['Year'] = df6.loc[:, 'NewDateTime'].dt.year
df6['Week'] = df6.loc[:, 'NewDateTime'].dt.isocalendar().week
test = np.array(df6.groupby(['Year', 'Week']), dtype = object).T
#排除⻑度不為168的array
Array = np.empty([1, 168])
for i in range(0, len(np.array(test[1]))):
    if len(np.array(test[1][i]['close'])) == 168 :
        Array = np.append(Array, np.array(test[1][i]['close']).reshape(1, -1), axis = 0)

AttributeError: 'DatetimeProperties' object has no attribute 'isocalendar'

In [9]:
df6.head()

,close,open,high,low,Volume BTC,NewDateTime,NewDate,returns,log_returns,Year
40422,4315.32,4308.83,4328.69,4291.37,23.2300,2017-08-17 05:00:00,2017/8/17,1.001506,0.001505,2017
40421,4324.35,4315.32,4345.45,4309.37,7.2300,2017-08-17 06:00:00,2017/8/17,1.002093,0.002090,2017
40420,4349.99,4324.35,4349.99,4287.41,4.4400,2017-08-17 07:00:00,2017/8/17,1.005929,0.005912,2017
40419,4360.69,4349.99,4377.85,4333.32,0.9499,2017-08-17 08:00:00,2017/8/17,1.002460,0.002457,2017
40418,4444.00,4360.69,4445.78,4360.00,10.7600,2017-08-17 09:00:00,2017/8/17,1.019105,0.018925,2017


In [10]:
# 加入SOPR
df_SOPR = pd.read_csv('./sopr-btc-24h.csv', date_parser = True)
df_SOPR = df_SOPR.reindex(-np.sort(-np.arange(len(df_SOPR))))
df_SOPR.tail()
df_SOPR2 = df_SOPR.loc[:, ('t','v')] ; del df_SOPR ; gc.collect()
#切出⽇期時間欄位，符號統⼀
df_SOPR2['NewDate'] = df_SOPR2.loc[:, 't'].str.slice(start = 0, stop = 10).str.replace('-', '/').str.replace('/0', '/').str.rsplit(pat=' ', n=1).apply(lambda x : x[0])
df_SOPR2 = df_SOPR2[['v','NewDate']].drop_duplicates()
merge_SOPR = pd.merge(df6,df_SOPR2, on='NewDate')
merge_SOPR = merge_SOPR.rename(columns = {"v":"SOPR"})
len(merge_SOPR)


19599

In [11]:
merge_SOPR['Hour'] = merge_SOPR.loc[:, 'NewDateTime'].dt.hour
merge_SOPR['three'] = merge_SOPR['Hour'].apply(lambda x: 0 if x<8 else (8 if x<16 else 16))
merge_SOPR['three'] = merge_SOPR['Hour'].apply(lambda x: '0' if x<8 else ('8' if x<16 else '16'))
merge_SOPR['8h'] =  merge_SOPR["NewDate"] + ":" + merge_SOPR["three"]
merge_SOPR= merge_SOPR.drop(['Hour', 'three'], axis=1)
merge_SOPR

,close,open,high,low,Volume BTC,NewDateTime,NewDate,returns,log_returns,Year,SOPR,8h
0,7177.02,7195.23,7196.25,7175.46,480.93000,2020-01-01 00:00:00,2020/1/1,0.997469,-0.002534,2020,0.997801,2020/1/1:0
1,7216.27,7177.02,7230.00,7175.71,836.51000,2020-01-01 01:00:00,2020/1/1,1.005469,0.005454,2020,0.997801,2020/1/1:0
2,7242.85,7216.27,7244.87,7211.41,626.60000,2020-01-01 02:00:00,2020/1/1,1.003683,0.003677,2020,0.997801,2020/1/1:0
3,7225.01,7242.85,7245.00,7220.00,748.76000,2020-01-01 03:00:00,2020/1/1,0.997537,-0.002466,2020,0.997801,2020/1/1:0
4,7217.27,7225.01,7230.00,7215.03,442.09000,2020-01-01 04:00:00,2020/1/1,0.998929,-0.001072,2020,0.997801,2020/1/1:0
5,7224.21,7217.27,7229.76,7216.65,331.48000,2020-01-01 05:00:00,2020/1/1,1.000962,0.000961,2020,0.997801,2020/1/1:0
6,7225.62,7224.21,7236.27,7221.51,592.82000,2020-01-01 06:00:00,2020/1/1,1.000195,0.000195,2020,0.997801,2020/1/1:0
7,7209.83,7225.62,7232.94,7199.11,600.11000,2020-01-01 07:00:00,2020/1/1,0.997815,-0.002188,2020,0.997801,2020/1/1:0
8,7200.64,7209.83,7210.00,7180.00,870.90000,2020-01-01 08:00:00,2020/1/1,0.998725,-0.001275,2020,0.997801,2020/1/1:8
9,7188.77,7200.64,7210.51,7188.00,597.70000,2020-01-01 09:00:00,2020/1/1,0.998352,-0.001650,2020,0.997801,2020/1/1:8


In [16]:
# 加入Funding rate
df_fr = pd.read_csv('./Funding Rate History_BTCUSDT Perpetual_2021-12-11.csv', date_parser = True)
df_fr = df_fr.reindex(-np.sort(-np.arange(len(df_fr))))
df_fr2 = df_fr.loc[:, ('Time','Funding Rate')] ; del df_fr ; gc.collect()
df_fr2['NewDate'] = df_fr2.loc[:, 'Time'].str.slice(start = 0, stop = 10).str.replace('-', '/').str.replace('/0', '/').str.rsplit(pat=' ', n=1).apply(lambda x : x[0])
# df_fr2 = df_fr2[['Funding Rate','NewDate']].drop_duplicates()


df_fr2['NewDateTime'] = df_fr2['Time'].apply(lambda x: pd.to_datetime(x, format = "%Y/%m/%d %H:%M"))
df_fr2['Hour'] = df_fr2.loc[:, 'NewDateTime'].dt.hour
df_fr2['three'] = df_fr2['Hour'].apply(lambda x: '0' if x<8 else ('8' if x<16 else '16'))
df_fr2['8h'] =  df_fr2["NewDate"] + ":" + df_fr2["three"]
df_fr2= df_fr2.drop(['Time','NewDate','NewDateTime','Hour', 'three'], axis=1)
df_fr2['fundingRate'] = df_fr2.loc[:, 'Funding Rate'].str.replace('%', '')
df_fr2 = df_fr2.drop(['Funding Rate'], axis=1)

merge_fr = pd.merge(merge_SOPR,df_fr2, on='8h')
merge_fr.head()

,close,open,high,low,Volume BTC,NewDateTime,NewDate,returns,log_returns,Year,SOPR,8h,fundingRate
0,7177.02,7195.23,7196.25,7175.46,480.93,2020-01-01 00:00:00,2020/1/1,0.997469,-0.002534,2020,0.997801,2020/1/1:0,-0.0025
1,7216.27,7177.02,7230.00,7175.71,836.51,2020-01-01 01:00:00,2020/1/1,1.005469,0.005454,2020,0.997801,2020/1/1:0,-0.0025
2,7242.85,7216.27,7244.87,7211.41,626.60,2020-01-01 02:00:00,2020/1/1,1.003683,0.003677,2020,0.997801,2020/1/1:0,-0.0025
3,7225.01,7242.85,7245.00,7220.00,748.76,2020-01-01 03:00:00,2020/1/1,0.997537,-0.002466,2020,0.997801,2020/1/1:0,-0.0025
4,7217.27,7225.01,7230.00,7215.03,442.09,2020-01-01 04:00:00,2020/1/1,0.998929,-0.001072,2020,0.997801,2020/1/1:0,-0.0025


In [17]:
# 加入google trend
df_GT = pd.read_csv('./googleTrend.csv', date_parser = True)
df_GT = df_GT.reindex(-np.sort(-np.arange(len(df_GT))))
df_GT.tail()
df_GT2 = df_GT.loc[:, ('date','bitcoin')] ; del df_GT ; gc.collect()

df_GT2['newdate']=df_GT2.loc[:, 'date'].str.slice(start = 0, stop = 19)
#切出⽇期時間欄位，符號統⼀
df_GT2['NewDateTime'] = df_GT2['newdate'].apply(lambda x: pd.to_datetime(x, format = "%Y/%m/%d %H"))
df_GT2 = df_GT2.drop(['date','newdate'], axis=1)
df_GT2.tail()
merge_GT = pd.merge(merge_fr,df_GT2, on='NewDateTime')
merge_GT = merge_GT.rename(columns = {"bitcoin":"googleTrend"})
merge_GT.head()


,close,open,high,low,Volume BTC,NewDateTime,NewDate,returns,log_returns,Year,SOPR,8h,fundingRate,googleTrend
0,8431.82,8145.28,8455.00,8142.00,11041.87,2020-01-08 00:00:00,2020/1/8,1.035179,0.034574,2020,1.011291,2020/1/8:0,0.0100,89
1,8370.92,8431.82,8448.14,8328.89,7355.82,2020-01-08 01:00:00,2020/1/8,0.992777,-0.007249,2020,1.011291,2020/1/8:0,0.0100,86
2,8278.59,8370.92,8422.00,8256.74,4048.91,2020-01-08 02:00:00,2020/1/8,0.988970,-0.011091,2020,1.011291,2020/1/8:0,0.0100,75
3,8300.34,8278.59,8344.00,8244.93,3947.38,2020-01-08 03:00:00,2020/1/8,1.002627,0.002624,2020,1.011291,2020/1/8:0,0.0100,68
4,8344.63,8300.34,8355.87,8288.00,2374.55,2020-01-08 04:00:00,2020/1/8,1.005336,0.005322,2020,1.011291,2020/1/8:0,0.0100,68


In [18]:
# save updated dataset
merge_GT = merge_GT.drop(['NewDate','Year','8h'], axis=1)
merge_GT.head()



,close,open,high,low,Volume BTC,NewDateTime,returns,log_returns,SOPR,fundingRate,googleTrend
0,8431.82,8145.28,8455.00,8142.00,11041.87,2020-01-08 00:00:00,1.035179,0.034574,1.011291,0.0100,89
1,8370.92,8431.82,8448.14,8328.89,7355.82,2020-01-08 01:00:00,0.992777,-0.007249,1.011291,0.0100,86
2,8278.59,8370.92,8422.00,8256.74,4048.91,2020-01-08 02:00:00,0.988970,-0.011091,1.011291,0.0100,75
3,8300.34,8278.59,8344.00,8244.93,3947.38,2020-01-08 03:00:00,1.002627,0.002624,1.011291,0.0100,68
4,8344.63,8300.34,8355.87,8288.00,2374.55,2020-01-08 04:00:00,1.005336,0.005322,1.011291,0.0100,68


In [19]:
merge_GT.to_csv('bitcoin_data.csv')